# **Name - PRABHAKAR KUMAR**
# **Roll - IRM2017008**
# **Naive Bayes Assignment Part 1**

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import string

#**Loading Dataset and preparing Testing and Training Data**

In [3]:
df = pd.read_csv("spam ham data set.csv",encoding='latin-1')
df.dropna(axis=1, inplace=True)
df.head

<bound method NDFrame.head of         v1                                                 v2
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]>

In [5]:
row_count = len(df.index)
train_count = int(0.7*row_count)
print(train_count)

3900


In [6]:
trainSet = df[:3900]
testSet = df[3900:]
testSet.reset_index(inplace=True,drop=True)

# **Calculating CLass Prior Probability and Likelihood**

In [8]:
spamCount = 0
hamCount = 0
totalCount =  0
spamDict = defaultdict(int)
hamDict = defaultdict(int)
for index,row in trainSet.iterrows():
    totalCount+=1

    # Splitting each text content into list of words
    wordList = str(row['v2']).split()

    # Removing Punctuations marks and lower casing from the words as they have no significance towards the model
    wordList = [(word.translate(str.maketrans('', '', string.punctuation))).lower() for word in wordList]

    # Removing empty words, as some empty words after removal of punctuations may have occured
    wordList = [word for word in wordList if word != '']

    if str(row['v1'])=='spam':
        spamCount+=1
        tempDict = spamDict
    else:
        hamCount+=1
        tempDict = hamDict
    for word in wordList:
        tempDict[word] = tempDict[word]+1

print(totalCount)
print(spamCount)
print(hamCount)

3900
519
3381


In [10]:
pSpam = spamCount/totalCount
print(pSpam)
pHam = hamCount/totalCount
print(pHam)

0.13307692307692306
0.8669230769230769


# **Applying Laplace Smoothing**

In [11]:
alpha = 1
k = 2
pWordSpamDict = {}
pWordHamDict = {}
for key in spamDict:
    pWordSpamDict[key] = (spamDict[key] + alpha)/(spamCount + (alpha*k))
for key in hamDict:
    pWordHamDict[key] = (hamDict[key] + alpha)/(hamCount + (alpha*k))

# **Probability of Mail being Spam using Laplace Smoothing**

In [12]:
def spamProb(wordList):
    wordSpamProb = []
    for word in wordList:
        if word in pWordSpamDict:
            wordSpamProb.append(pWordSpamDict[word])
        else:
            p = (alpha)/(spamCount + alpha*k)
            wordSpamProb.append(p)
    prob = pSpam
    for item in wordSpamProb:
        prob*=item
    return prob

# **Probability of Mail being Ham using Laplace Smoothing**

In [13]:
def hamProb(wordList):
    wordHamProb = []
    for word in wordList:
        if word in pWordHamDict:
            wordHamProb.append(pWordHamDict[word])
        else:
            p = (alpha)/(hamCount + alpha*k)
            wordHamProb.append(p)
    prob = pHam
    for item in wordHamProb:
        prob*=item
    return prob

# **Testing of Model over Test Data**

In [14]:
actual = np.zeros(len(testSet.index), dtype=int)
predicted = np.zeros(len(testSet.index), dtype=int)
for index, row in testSet.iterrows():
    if str(row['v1'])=='spam':
        actual[index] = 1
    else:
        actual[index] = 0
    
    mail = str(row['v2'])
    wordList = str(mail).split()
    wordList = [(word.translate(str.maketrans('', '', string.punctuation))).lower() for word in wordList]
    wordList = [word for word in wordList if word != '']
    
    pSpamMail = spamProb(wordList)
    pHamMail = hamProb(wordList)
    
    if pSpamMail > pHamMail:
        predicted[index] = 1
    else:
        predicted[index] = 0
        
print('TOTAL SAMPLES : ' + str(len(testSet.index)))
print('SAMPLES CLASSIFIED CORRECTLY : ' + str(np.sum(actual == predicted)))
print('SAMPLES CLASSIFIED INCORRECTLY : ' + str(len(testSet.index) - np.sum(actual == predicted)))
print('MODEL ACCURACY : ' + str(((np.sum(actual == predicted))/(len(testSet.index)))*100))

TOTAL SAMPLES : 1672
SAMPLES CLASSIFIED CORRECTLY : 1271
SAMPLES CLASSIFIED INCORRECTLY : 401
MODEL ACCURACY : 76.01674641148325


# **Accuracy Parameters of the Model over Test Data**

In [17]:
tp=0
tn=0
fp=0
fn=0
for i in range(1672):
    if actual[i]==predicted[i]and actual[i]==0:
        tp+=1
    if actual[i]!=predicted[i]and actual[i]==0:
        fp+=1
    if actual[i]==predicted[i]and actual[i]==1:
        tn+=1
    if actual[i]!=predicted[i]and actual[i]==1:
        fn+=1

print("CONFUSION MATRIX")
print(tp," ",fp)
print(fn," ",tn)

CONFUSION MATRIX
1043   401
0   228


In [20]:
print("Accuracy : ",str((tp+tn)/(tp+fp+tn+fn)))
print("Precision : ",str((tp)/(tp+fp)))
print("Sensitivity : ",str((tp)/(tp+fn)))
print("Specificity : ",str((tn)/(fp+tn)))

Accuracy :  0.7601674641148325
Precision :  0.7222991689750693
Sensitivity :  1.0
Specificity :  0.3624801271860095
